<a href="https://colab.research.google.com/github/stavco9/nlp-final-project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/stavco9/nlp-final-project.git

Cloning into 'nlp-final-project'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 236 (delta 109), reused 161 (delta 59), pack-reused 0
Receiving objects: 100% (236/236), 9.07 MiB | 7.72 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [2]:
%cd /content/nlp-final-project/

/content/nlp-final-project


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using c

In [4]:
import os
import json
import pandas as pd
import numpy as np
import requests

In [5]:
# Mount Google Drive
from google.colab import drive, files
drive.mount('/content/drive')
!ln -s "/content/drive/My Drive/IDC_MSc/Year1/NLP/Final-Project/data" ./data

Mounted at /content/drive


In [ ]:
# Please set a valid Google API
# Please download the 'data' folder from https://drive.google.com/drive/folders/1D-OHX0LUNA5Y-0h8UXwqa4H_yETZN_t_ to your project root directory

GOOGLE_API_KEY = '**********************'

In [ ]:
##
## Dataset preperation part
##

In [ ]:
from modules.dataset.transcript_dataset import TranscriptDataset

dataset_builder = TranscriptDataset(api_key=GOOGLE_API_KEY)

# dataset_folder => The folder to load / save the datasets file from. If not exists it will create it automatically
# videos_count => Total number of videos to include in the dataset. If there are already saved videos in the "data" folder,
# it will fill the remaining videos count (For example if the parameter: videos_count is 4000 and there are already 1500 videos
# in the folder, it will fetch 2500 videos). Default is 100
# country_code => 2 digits ISO Code of the country you want to fetch videos from (If not set, the default is United States 'us')
# Relevant codes for countries with a majority of English based videos => 'ca' Canada, 'uk' United Kingdom, 'au' Australia, 'in' India
#  'nz' New Zealand, 'za' South Africa
# language => 2 digits ISO Code of the video's language you want to use (If not set, the default is English 'en')
dataset_builder.build_dataset(dataset_folder='data/dataset', videos_count=5000, country_code='uk')

# dataset_folder => The folder to load / save the datasets file from. If not exists it will create it automatically
# comments_count => The amount of comments to fetch for each video. Default is 100
dataset_builder.build_comments(dataset_folder='data', comments_count=200)

There are currnet 5000 videos in 10 files
There are already at least 5000. No need to do anything


In [ ]:
##
## Sentiment analysis part
##

In [ ]:
from modules.sentiment_analysis.comments_sentiment_analysis import multi_sentiment_score
from modules.sentiment_analysis.video_info import Video_info, datumToObj, save_to_json, save_to_pickle
from modules.sentiment_analysis.GPT_comment_controversy import init_openai, get_disagreement_rating
from modules.sentiment_analysis.scoring_video import adjust_disagreement_rating

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# put all the json into pickle
datasets = [f'data/dataset/dataset_{i}.json' for i in range(10)]
to_print = True #input("Do you want to show prints? (y/n): ") != 'n'
basic_inf_vids = datumToObj(datasets, 'data/sentiment/videos_sample.pkl', to_print)
number_of_vids = len(basic_inf_vids)

save_to_json(basic_inf_vids, 'data/sentiment/vids_no_sentiment.json')
sentiment_model = multi_sentiment_score
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
count = 0
for vid in basic_inf_vids:
    print_text = "\r"
    if to_print:
        count += 1
        print_text += f"Processing video {count}/{number_of_vids}"
    vid.add_comments_scores(sentiment_model, model_name)
    if to_print:
        print_text += f" - Disagreement rating: {vid.comments_controversy_classical}"
    if to_print:
        print(print_text)

save_to_json(basic_inf_vids, 'data/sentiment/vids_classical_sentiment.json')
basic_inf_vids = adjust_disagreement_rating(basic_inf_vids)
save_to_json(basic_inf_vids, 'data/sentiment/videos_classical_sentiment_adjusted.json')
'''

#load vids from videos_classical_sentiment_adjusted.json and adjust basic_inf_vids with its values
with open('data/data/videos_classical_sentiment_adjusted.json') as json_file:
    json_obj = json.load(json_file)
for i, video_obj in enumerate(json_obj):
    basic_inf_vids[i].comments_controversy_classical = float(video_obj['comments_controversy_classical'])
'''


to_continue = True #input("Do you want to continue with GPT? (y/n): ")
if not to_continue:
  print("Done")
else:
  count = 0
  client = init_openai()
  sentiment_model=get_disagreement_rating
  model_name="GPT"
  for vid in basic_inf_vids:
      print_text = "\r"
      if to_print:
          count += 1
          print_text += f"Processing video {count}/{number_of_vids}"
      vid.add_comments_scores(sentiment_model, model_name, client)
      if to_print:
          print_text += f" - Disagreement rating: {vid.comments_controversy_GPT}"
      if to_print:
          print(print_text)
  save_to_json(basic_inf_vids, 'data/data/vids_GPT_sentiment.json')

In [ ]:
##
## Training part
##

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import random
import numpy as np
import json
import os
from modules.azsclm.azsclm import AZSC_LanguageModel
from modules.azsclm.board import Board
from transformers import AutoTokenizer

In [15]:
class Config_Model:
    def __init__(self, num_epochs, train_data, valid_data, test_data, model, optimizer, threshold_percentage, criterion, device):
        self.num_epochs = num_epochs
        self.train_data = train_data
        self.valid_data = valid_data
        self.test_data = test_data
        self.device = device
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.threshold_percentage = threshold_percentage


class Model:
    def __init__(self, config, data_name = 'GPT'):
        self.num_epochs = config.num_epochs
        self.train_data = config.train_data
        self.valid_data = config.valid_data
        self.test_data = config.test_data
        self.pred_score = []
        self.true_score = []
        self.device = config.device
        self.model = config.model
        self.optimizer = config.optimizer
        self.criterion = config.criterion
        self.data_name = 'comments_controversy_' + data_name
        self.max_available = 1
        self.min_available = -1
        self.board = Board()

    def calculate_accuracy(self, min_item, max_item, pred, true):
        pred = max(pred, self.min_available)
        pred = min(pred, self.max_available)
        true = max(true, self.min_available)
        true = min(true, self.max_available)

        acc = 1 - (abs(pred - true) / abs(max_item - min_item))
        return float(acc)

    def train(self):
        size = len(self.train_data.dataset)
        num_train_batches = len(self.train_data)
        num_valid_batches = len(self.valid_data)
        self.model.train()
        for epoch in range(self.num_epochs):
            print(f"Epoch {epoch+1}\n-------------------------------")
            epoch_loss = 0

            for batch_idx, (src, trg) in enumerate(self.train_data):
                src, trg = src.to(self.device), trg.to(self.device)
                output = self.model(src)

                self.optimizer.zero_grad()
                loss = self.criterion(output, trg)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.5)
                train_loss = loss.item()
                self.optimizer.step()
                epoch_loss += train_loss

            self.board.info_handler(loss=train_loss, batch=batch_idx, size=size, epoch_loss=epoch_loss, name='(training) ' + self.data_name)

            print("Evaluating with validation set")
            self.model.eval()
            running_valid_loss = 0
            total_acc = 0

            list_targets = [item[1].to('cuda') for item in self.valid_data]
            list_targets = torch.stack(list_targets, dim=1).to('cuda')
            max_item = min(torch.max(list_targets).item(), self.max_available)
            min_item = max(torch.min(list_targets).item(), self.min_available)

            # Disable gradient computation and reduce memory consumption.
            with torch.no_grad():
                for batch_idx, (src, trg) in enumerate(self.valid_data):
                    src, trg = src.to(self.device), trg.to(self.device)
                    output = self.model(src)
                    loss = self.criterion(output, trg)
                    valid_loss = loss.item()
                    running_valid_loss += valid_loss
                    total_acc += self.calculate_accuracy(max_item, min_item, output, trg)
                    avg_acc = total_acc / (batch_idx + 1)
                    print(f"Accuracy: {avg_acc}")

            avg_train_loss = epoch_loss / num_train_batches
            avg_valid_loss = running_valid_loss / num_valid_batches
            print(f"Average train loss: {avg_train_loss}, avarage valid loss: {avg_valid_loss}")
            avg_acc = total_acc / num_valid_batches
            print(f"Average accuracy rate: {avg_acc}")

        batch = next(iter(self.train_data))
        src = batch[0].to(self.device)
        self.board.add_graph(self.model, src)

    def test(self):
        size = len(self.test_data.dataset)
        num_batches = len(self.test_data)
        self.model.eval()
        epoch_loss = 0.0
        sections_losses = {}
        total_acc = 0

        list_targets = [item[1].to('cuda') for item in self.test_data]
        list_targets = torch.stack(list_targets, dim=1).to('cuda')
        max_item = min(torch.max(list_targets).item(), self.max_available)
        min_item = max(torch.min(list_targets).item(), self.min_available)
        with torch.no_grad():
            for batch_idx, (src, trg) in enumerate(self.test_data):
                src, trg = src.to(self.device), trg.to(self.device)
                pred = self.model(src)
                pred = pred.to(self.device)
                loss = self.criterion(pred, trg)
                #for section_name, section_loss in loss.last_losses.items():
                #    if section_name not in sections_losses:
                #        sections_losses[section_name] = torch.Tensor([0]).to(self.device)
                #    sections_losses[section_name] += section_loss
                #for section_name in sections_losses.keys():
                #    sections_losses[section_name] /= size
                epoch_loss = self.board.info_handler(loss=loss.item(), batch=batch_idx, size=size, epoch_loss=epoch_loss, name = '(test) ' + self.data_name)
                avg_loss = epoch_loss / num_batches
                print(f"Average loss: {avg_loss}")
                total_acc += self.calculate_accuracy(max_item, min_item, pred, trg)
                avg_acc = total_acc / (batch_idx + 1)
                print(f"Accuracy: {avg_acc}")
                self.pred_score.extend(pred.cpu().numpy())
                self.true_score.extend(trg.cpu().numpy())

        avg_loss = epoch_loss / num_batches
        print(f"Average loss rate: {avg_acc}")

        avg_acc = total_acc / num_batches
        print(f"Average accuracy rate: {avg_acc}")

    def calculate_metrics(self):
      # Convert lists to tensors for calculation
      true_score_tensor = torch.tensor(self.true_score)
      pred_score_tensor = torch.tensor(self.pred_score)

      # Calculating precision, recall, and F1 score using PyTorch
      TP = ((pred_score_tensor >= 0) & (true_score_tensor >= 0)).sum().item()
      FP = ((pred_score_tensor >= 0) & (true_score_tensor < 0)).sum().item()
      FN = ((pred_score_tensor < 0) & (true_score_tensor >= 0)).sum().item()

      precision = TP / (TP + FP) if TP + FP > 0 else 0
      recall = TP / (TP + FN) if TP + FN > 0 else 0
      f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

      print(f'Precision: {precision}')
      print(f'Recall: {recall}')
      print(f'F1 Score: {f1}')

      return (precision, recall, f1)



In [16]:
def custom_collate_fn(batch):
    src, tgt = zip(*batch)
    src = torch.stack(src).long()  # Ensure src is Long
    src = src.view(src.size(0), -1)
    tgt = torch.stack(tgt).float()  # Ensure tgt is Float
    tgt = tgt.view(tgt.size(0), -1)
    return src, tgt

class CustomDataset(Dataset):
    def __init__(self, texts, scores):
        self.texts = texts
        self.scores = scores

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        src = self.texts[idx]
        trg = self.scores[idx]
        return src, trg

def homogonize_token_length(text, tokenizer, text_length, device="cuda"):
    tokenized_text = tokenizer(text, return_tensors="pt", truncation=True, padding=True)['input_ids'].long().to(device)
    if tokenized_text.shape[1] < text_length:
        tokenized_text = torch.cat((tokenized_text, torch.zeros(tokenized_text.shape[0], text_length - tokenized_text.shape[1]).to(device)), dim=1)
    else:
        tokenized_text = tokenized_text[:, :text_length]
    return tokenized_text

class Datasets:
    def __init__(self, json_file, tokenizer, text_length, ratios=[0.8, 0.1, 0.1], batch_size=1, device="cuda"):
        self.json_file = json_file
        assert sum(ratios) == 1
        self.ratios = ratios
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.text_length = text_length
        self.device = device
        self.load_data()

    def load_data(self):
        with open(self.json_file, 'r') as f:
            data = json.load(f)

        random.shuffle(data)
        tokens = [homogonize_token_length(d[0], self.tokenizer, self.text_length, self.device) for d in data]
        GPT_scores = [torch.tensor(d[1], dtype=torch.float32).to(self.device) for d in data]
        BERT_scores = [torch.tensor(d[2], dtype=torch.float32).to(self.device) for d in data]

        print(BERT_scores)

        train_size = int(self.ratios[0] * len(data))
        val_size = int(self.ratios[1] * len(data))
        #test_size = len(data) - train_size - val_size

        train_text = tokens[:train_size]
        val_text = tokens[train_size:train_size + val_size]
        test_text = tokens[train_size + val_size:]

        train_scores1 = GPT_scores[:train_size]
        val_scores1 = GPT_scores[train_size:train_size + val_size]
        test_scores1 = GPT_scores[train_size + val_size:]

        train_scores2 = BERT_scores[:train_size]
        val_scores2 = BERT_scores[train_size:train_size + val_size]
        test_scores2 = BERT_scores[train_size + val_size:]

        train_dataset1 = CustomDataset(train_text, train_scores1)
        val_dataset1 = CustomDataset(val_text, val_scores1)
        test_dataset1 = CustomDataset(test_text, test_scores1)

        train_dataset2 = CustomDataset(train_text, train_scores2)
        val_dataset2 = CustomDataset(val_text, val_scores2)
        test_dataset2 = CustomDataset(test_text, test_scores2)

        # Create data loaders
        self.train_loader1 = DataLoader(train_dataset1, batch_size=self.batch_size, shuffle=True, collate_fn=custom_collate_fn)
        self.val_loader1 = DataLoader(val_dataset1, batch_size=self.batch_size, shuffle=False, collate_fn=custom_collate_fn)
        self.test_loader1 = DataLoader(test_dataset1, batch_size=self.batch_size, shuffle=False, collate_fn=custom_collate_fn)

        self.train_loader2 = DataLoader(train_dataset2, batch_size=self.batch_size, shuffle=True, collate_fn=custom_collate_fn)
        self.val_loader2 = DataLoader(val_dataset2, batch_size=self.batch_size, shuffle=False, collate_fn=custom_collate_fn)
        self.test_loader2 = DataLoader(test_dataset2, batch_size=self.batch_size, shuffle=False, collate_fn=custom_collate_fn)

In [17]:
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
    torch.cuda.empty_cache()

    print(torch.cuda.memory_summary(device="cuda", abbreviated=True))

    text_length = 4096
    tokenizer_name = 'distilbert-base-uncased-finetuned-sst-2-english'
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    datasets = Datasets("data/sentiment/vids_sentiment.json", tokenizer, text_length, ratios=[0.8, 0.1, 0.1], batch_size=1, device="cuda")

    # Instantiate the model
    model = AZSC_LanguageModel(tokenizer, text_length).to("cuda")

    # Instantiate the optimizer
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)

    # Instantiate the loss function
    criterion = nn.MSELoss()

    # the gpt model
    traind_data_gpt = datasets.train_loader1
    valid_data_gpt = datasets.val_loader1
    test_data_gpt = datasets.test_loader1

    config_gpt = Config_Model(num_epochs=150, train_data=traind_data_gpt, valid_data=valid_data_gpt, test_data=test_data_gpt, model=model, optimizer=optimizer, threshold_percentage = 5, criterion=criterion, device="cuda")
    # Instantiate the Train object
    model_gpt = Model(config_gpt, data_name='GPT')

    # the bert model
    trained_data_classical = datasets.train_loader2
    valid_data_classical = datasets.val_loader2
    test_data_classical = datasets.test_loader2
    config_classical = Config_Model(num_epochs=2, train_data=trained_data_classical, valid_data=valid_data_classical, test_data=test_data_classical, model=model, optimizer=optimizer, threshold_percentage = 5, criterion=criterion, device="cuda")
    # Instantiate the Train object
    model_classical = Model(config_classical, data_name='classical')


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 161903 KiB |   2560 MiB |  49115 GiB |  49115 GiB |
|---------------------------------------------------------------------------|
| Active memory         | 161903 KiB |   2560 MiB |  49115 GiB |  49115 GiB |
|---------------------------------------------------------------------------|
| Requested memory      | 157840 KiB |   2556 MiB |  49110 GiB |  49110 GiB |
|---------------------------------------------------------------

In [13]:
#for batch_idx, (src, trg) in enumerate(test_data_classical):
#    src, trg = src.to('cuda'), trg.to('cuda')

src, trg = next(iter(test_data_classical))
print(src)
#targets = [item[1].to('cuda') for item in test_data_classical]
#targets[5]
#targets = torch.stack(targets, dim=1).to('cuda')
#max_item = min(torch.max(targets).item(), 1)
#print(type(max_item))
#min_item = max(torch.min(targets).item(), -1)
#print(type(min_item))
#print(abs(max_item - min_item) == 2)

tensor([[ 101, 3100, 2061,  ...,    0,    0,    0]], device='cuda:0')


In [18]:
    # Start training
    model_classical.train()

Epoch 1
-------------------------------
Evaluating with validation set
Accuracy: 0.9501653909683228
Accuracy: 0.9260447919368744
Accuracy: 0.9218974709510803
Accuracy: 0.9030338674783707
Accuracy: 0.901070523262024
Accuracy: 0.7847260038057963
Accuracy: 0.7806957534381321
Accuracy: 0.8077205792069435
Accuracy: 0.8248839444584317
Accuracy: 0.8349178791046142
Accuracy: 0.8405972502448342
Accuracy: 0.8479097137848536
Accuracy: 0.8582583574148325
Accuracy: 0.8612334430217743
Accuracy: 0.867395015557607
Accuracy: 0.8713153526186943
Accuracy: 0.8711700158960679
Accuracy: 0.875432292620341
Accuracy: 0.8800925549707914
Accuracy: 0.8850479781627655
Accuracy: 0.8848284993852887
Accuracy: 0.8840539753437042
Accuracy: 0.8872750090516132
Accuracy: 0.8827793424328169
Accuracy: 0.8824328827857971
Accuracy: 0.885026060617887
Accuracy: 0.8855770848415516
Accuracy: 0.8853970659630639
Accuracy: 0.8848594632642023
Accuracy: 0.8864686846733093
Accuracy: 0.8867883528432539
Accuracy: 0.8879944961518049
Accur

In [19]:
    model_classical.test()
    precision, recall, f1 = model_classical.calculate_metrics()

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')

Average loss: 3.114566434083915e-05
Accuracy: 0.9431480765342712
Average loss: 0.00013572279500161728
Accuracy: 0.919486403465271
Average loss: 0.0002765996994950422
Accuracy: 0.9060205618540446
Average loss: 0.00031793669275030856
Accuracy: 0.9131413847208023
Average loss: 0.000320712450562363
Accuracy: 0.9271186709403991
Average loss: 0.0003527572137167359
Accuracy: 0.929654449224472
Average loss: 0.0005064770292558866
Accuracy: 0.9216606276375907
Average loss: 0.0005143812411782763
Accuracy: 0.9278730303049088
Average loss: 0.008520119499540093
Accuracy: 0.8427599536048042
Average loss: 0.008726194799343896
Accuracy: 0.8438601911067962
Average loss: 0.008733672764799671
Accuracy: 0.8555222424593839
Average loss: 0.010121069001544993
Accuracy: 0.8359418014685313
Average loss: 0.010223862390522278
Accuracy: 0.8406168130727915
Average loss: 0.010268230861473101
Accuracy: 0.8471545236451286
Average loss: 0.010970250369765138
Accuracy: 0.8393501440684
Average loss: 0.010976293336361556
A

<ipython-input-15-7ffc81f5f320>:136: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  true_score_tensor = torch.tensor(self.true_score)


In [20]:
def print_tensor(tensor_arr):
  true_score_items = [float("{:.2f}".format(x.item())) for x in tensor_arr]
  start = 0
  end = len(true_score_items)
  step = 20
  for i in range(start, end, step):
      x = i
      print(true_score_items[x:x+step])

true_score_tensor = torch.tensor(model_classical.true_score)
pred_score_tensor = torch.tensor(model_classical.pred_score)

print("True values\n-----------")
print_tensor(true_score_tensor)
print("Pred values\n-----------")
print_tensor(pred_score_tensor)

True values
-----------
[-0.78, -0.87, -0.91, -0.8, -0.7, -0.78, -0.41, -0.72, 1.15, -0.96, -0.72, 0.09, -0.87, -0.8, -0.13, -0.72, -0.76, -0.71, -0.71, -0.87]
[0.57, -0.51, -0.76, -0.81, -0.84, -0.92, -0.75, 0.01, -0.8, -0.86, -0.83, -0.32, -0.74, -0.74, -0.81, -0.13, -0.79, -0.61, -0.57, -0.71]
[-0.94, -0.85, -0.76, -0.74, -0.88, -0.91, -0.76, -0.92, -0.7, -0.74, 6.28, -0.83, -0.76, -0.74, -0.73, 2.46, 1.04, -0.78, -0.52, 0.82]
[-0.82, -0.78, 0.58, 0.16, -0.66, -0.91, -0.9, -0.72, -0.72, -0.45, -0.95, -0.76, -0.32, -0.79, -0.73, 1.64, -0.85, -0.92, -0.44, -0.87]
[-0.73, -0.65, -0.86, -0.66, -0.33, -0.84, -0.62, -0.84, -0.79, -0.75, -0.77, -0.85, -0.26, -0.34, -0.74, -0.85, -0.78, -0.71, -0.65, -0.63]
[-0.88, -0.71, -0.78, -0.54, -0.65, -0.52, -0.87, -0.74, -0.85, -0.42, 1.71, -0.74, -0.71, -0.78, -0.7, -0.73, -0.73, -0.84, -0.95, -0.88]
[-0.42, -0.74, -0.81, -0.52, -0.7, -0.82, -0.64, -0.72, -0.73, -0.82, -0.91, -0.6, -0.66, -0.72, 0.12, -0.9, -0.77, -0.75, -0.82, 0.79]
[-0.84, -0.9,